In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline
COLOR_TREAT = "#2ecc71"
COLOR_NO_TREAT = "#e74c3c"
%config InlineBackend.figure_format = 'retina'

# Question 1: Propensity score matching

We will work with a dataset from Robert LaLonde's study Evaluating the Econometric Evaluations of Training Programs.

The study investigated the effect of a job training program ("National Supported Work Demonstration") on the real earnings of an individual, a couple of years after completion of the program.
Our task is to determine the effectiveness of the "treatment" represented by the job training program.

The features of this dataset are:

- `treat`: 1 if the subject participated in the job training program, 0 otherwise
- `age`: the subject's age
- `educ`: years of education
- `race`: categorical variable with three possible values: Black, Hispanic, or White
- `married`: 1 if the subject was married at the time of the training program, 0 otherwise
- `nodegree`: 1 if the subject has earned no school degree, 0 otherwise
- `re74`: real earnings in 1974 (pre-treatment)
- `re75`: real earnings in 1975 (pre-treatment)
- `re78`: real earnings in 1978 (outcome)


First let's import and have a look to the data :

Note that the `race` variable is changed to `black` and `hispan`. Therefore someone with `black=0` and `hispan=0` is White.

In [ ]:
#let's import the dataset and observe the first rows
lalonde = pd.read_csv('lalonde.csv')
lalonde.head()

## 1. Naive analysis

Compare the distribution of the outcome variable (`re78`) between the two groups, using plots and numbers.

In [ ]:
#We define this function to have clean distribution plot as we are going to use them a lot through our analysis
def plot_distrib(s1, s2, title, xLabel, yLabel, ax=None):
    bins = np.histogram(s1)[1]
    sns.distplot(s1, kde=False, color=COLOR_NO_TREAT, norm_hist=True, ax=ax, bins=bins)
    sns.distplot(s2, kde=False, color=COLOR_TREAT, norm_hist=True, ax=ax, bins=bins)
    if ax is None:
        plt.title(title)
        plt.xlabel(xLabel)
        plt.ylabel(yLabel)
        plt.legend(['No treatment', 'Treatment'])
    else:
        ax.set_title(title)
        ax.set_xlabel(xLabel)
        ax.set_ylabel(yLabel)
        ax.legend(['No treatment', 'Treatment'])

First, let's compare the two distributions. The distribution in red is the distribution of the revenues in 78 for the people that didn't take part in the job training program (no treatment), whereas the one in green is the distribution of the revenues in 78 for the people that took part in the job training program (treatment).

In [ ]:
#let's plot the two distributions
plt.figure(figsize=(12,5))
plot_distrib(s1=lalonde.re78[lalonde['treat'] == 0], s2=lalonde.re78[lalonde['treat'] == 1], title='Distribution of the revenues', ax=None, xLabel='Revenue in 1978', yLabel='Number of persons (Density)')

We observe that people that didn't take part in the training programs seem to have better earnings. Indeed, the histogram shows that the green bars are higher than the red ones for the small revenues whereas the red ones are higher for the higher revenues.

TODO : PEUT ETRE FAIRE UN KOLMO SMIRNORFF TEST ICI POUR MONTRER QUE LES DISTRIB SONT BIEN DIFFERENTES.

Now, let's compare other indicators for the people that have/have not participated the job training:

In [ ]:
lalonde['diff'] = lalonde['re78'] - lalonde['re75']

plt.figure(figsize=(20,5))
axes = plt.subplot(131)
lalonde.groupby(['treat'])['re78'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.title('Mean salary with/without training')
plt.ylabel('Mean salaray')
plt.xlabel('Traning')
plt.subplot(132)
lalonde.groupby(['treat'])['re78'].median().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.title('Median salary with/without training')
plt.ylabel('Median Salary')
plt.xlabel('Training')
plt.subplot(133)
lalonde.groupby(['treat'])['diff'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.title('Difference of salary between 78 and 75 with/without training')
plt.ylabel('Difference of Salary')
plt.xlabel('Training')

lalonde = lalonde.drop('diff',1)

Here we first compare the mean and the median of the revenues in 78 for the two groups. We notice that both of these indicators are higher for the people that didn't have the training. 

We thought that another interesting indicator would be to compare the difference of earnings between 78 and 75 for both groups. We observe on the plot that the training program helped a bit, but the difference is very small.

Therefore the naive approach concludes that the training program is not effective.

## 2. A closer look at the data

For each feature in the dataset, compare its distribution in the treated group with its distribution in the control group, using plots and numbers.

In [ ]:
# distinguish categorical from non-categorical features
sns.set(font_scale=1.2)
lalonde_cat = lalonde[['black', 'hispan', 'married', 'nodegree', 'treat']]
lalonde_non_cat = lalonde[['age', 'educ', 're74', 're75', 're78', 'treat']]

In [ ]:
f, axarr = plt.subplots(3, 3, figsize=(15, 18))
for index, feature in enumerate(['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']):
    ax = axarr[int(index/3)][index%3]
    plot_distrib(s1=lalonde[feature][lalonde['treat'] == 0], s2=lalonde[feature][lalonde['treat'] == 1], title=feature, xLabel = feature, yLabel='Density', ax=ax)

Here are all the distributions plotted. If we want our analysis not to be naive, these distributions should be almost the same along the two categories (treated and not treated).

We can already spot some huge differences from these plots (Features Black and Married for example) but some Kolmogorov–Smirnov 2 sample tests will help us be sure of our analysis.

The Kolmogorov–Smirnov 2 sample tests whether 2 samples are drawn from the same distribution. If the K-S statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same.

In [ ]:
from scipy import stats 
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\\n') 
for feature in ['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']: 
    ks = stats.ks_2samp(lalonde[feature][lalonde['treat'] == 0], lalonde[feature][lalonde['treat'] == 1]) 
    print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

Let's take a threshold of 0.05 for the p-value. We can therefore reject the null hypothesis for the following features :

- Age
- Black
- Married
- re74
- re75

As seen in class, a propensity score matching can help us equilibrate these distributions.

### Pairwise analysis of features' distribution
Comparing the distrbution of features for each distinct feature tuple can give additional information:
**TODO : décrire les obs**

In [ ]:
sns.pairplot(lalonde_non_cat, hue='treat', palette={0:"#e74c3c", 1: "#2ecc71"})

## 3. A propsensity score model

We should fit our model on the pre treatment features, though we will have to remove the re78 feature.
Furthermore, we preprocess our data by standardizing them so we don't have some features that have more importance than some others.

In [ ]:
import sklearn.linear_model
#removing the useless features
lal = lalonde.drop(['id','treat','re78'],1)
#Standardize the dataset
lal = preprocessing.scale(lal)
#Perform a logistic regression in order to get the propensity score for each individuals
model = sklearn.linear_model.LogisticRegression()
model.fit(lal, lalonde.treat)
pred = model.predict_proba(lal)

Here we check the performance of our model, we have an accuracy of 82%.

In [ ]:
sum(model.predict(lal) == lalonde.treat)/len(lal)

In [ ]:
plt.figure(figsize=(15, 10))
lalonde['pred'] = pred[:,1]
ax = sns.stripplot(x='id', y='pred', hue='treat', data=lalonde, palette={0:"#e74c3c", 1: "#2ecc71"})
ax.set(xticklabels=[], ylabel='Treatment (prediction)')
plt.title('Propensity score per induvidual')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., handles=ax.get_legend_handles_labels()[0], labels=['Without', 'With'], title='Treatment (ground truth)')

This plot shows us what is the propensity score for each individuals. 
The individuals that received the treatment are in green and should have a score around 1 whereas the individuals that didn't receive it are in red and should have a score around 0.

Even if we have some outliers, this is the case.

## 4. Balancing the dataset via matching
Use the propensity scores to match each data point from the treated group with exactly one data point from the control group, while ensuring that each data point from the control group is matched with at most one data point from the treated group.

In order to do this matching, we will procede in the following way:

- First we will create a bipartite graph where the first part is composed of people who received the training and the second part is composed of people who did not receive the training.

- Then we will make this bipartite graph complete. It means that the vertices are partitioned into two subsets V1 (the people who received the training) and V2 (the people who did not receive the traing) such that no edge has both endpoints in the same subset, and every possible edge that could connect vertices in different subsets is part of the graph. This way will will be sure to match people with treatment with people without treatmant

- We will give a weight to these edges: Let's take two individuals,we denote by $w_1$ the propensity score of the treated individual and $w_2$ the propensity score of the untreated individual. The weight of the edge between them will be:
$$W = - |w_{1} - w_{2}| $$

- Finally we will use a maximum weight matching with max cardinality algorithm. Note that we have a minus sign in our weights because we want to minimize the weights even though we are using a maximum weight matching algorithm. We are using the method implemented in the NetworkX package. This method is based on the “blossom” method for finding augmenting paths and the “primal-dual” method for finding a matching of maximum weight, you can read more about it there : https://dl.acm.org/citation.cfm?id=6502 .

In [ ]:
import networkx as nx
from networkx.algorithms import bipartite
#graph_size = lalonde['treat'].value_counts()
#G = nx.complete_bipartite_graph(graph_size[0], graph_size[1])
#G.add_nodes_from(lalonde['id'][lalonde.treat == 0], bipartite=0) # Add the node attribute "bipartite"
#G.add_nodes_from(lalonde['id'][lalonde.treat == 1], bipartite=1)

G=nx.Graph()
G.add_nodes_from(lalonde['id'][lalonde.treat == 0]) # Add the node attribute "bipartite"
G.add_nodes_from(lalonde['id'][lalonde.treat == 1])


In [ ]:
for ID_u, score_u in zip(lalonde.id[lalonde.treat == 0], lalonde.pred[lalonde.treat == 0]):
    for ID_v, score_v in zip(lalonde.id[lalonde.treat == 1], lalonde.pred[lalonde.treat == 1]):
        G.add_edge(ID_u, ID_v, weight=-abs(score_u-score_v))

In [ ]:
#G.number_of_nodes()
G.number_of_edges()
#nx.draw(G)

In [ ]:
from networkx.algorithms import max_weight_matching
matching = max_weight_matching(G, maxcardinality=True)

In [ ]:
res = dict()
for key in matching.keys():
    if(key[0] == 'N'):
        res[key] = matching[key]
res

In [ ]:
lalonde_treated = lalonde[lalonde['treat'] == 1]
lalonde_treated['temp'] = 1
#lalonde_treated

lalonde_nt = lalonde[lalonde['treat'] == 0]
lalonde_nt['temp'] = 1
#lalonde_nt

result = pd.merge(lalonde_treated, lalonde_nt, on='temp')
result = result[['id_x' , 'id_y' , 'pred_x' , 'pred_y']]
result['diff'] = abs(result['pred_x'] - result['pred_y'])
result = result.set_index(['id_x', 'id_y'])
sum(result.loc[list(res.items())]['diff'])


In [ ]:
result.loc[list(res.items())]

In [ ]:
lalondeid = lalonde.set_index('id')
matched = lalondeid.loc[list(matching.keys())]

In [ ]:
plot_distrib(s1=matched.re78[matched['treat'] == 0], s2=matched.re78[matched['treat'] == 1], 
             title='Distribution of the revenues', ax=None, xLabel='Revenue in 1978', yLabel='Number of persons (Density)')

In [ ]:
matched['diff'] = matched['re78'] - matched['re75']



plt.figure(figsize=(20,5))
plt.subplot(131)
plt.title('Mean salary with/without training')
plt.ylabel('Mean salaray')
matched.groupby(['treat'])['re78'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.subplot(132)
plt.title('Median salary with/without training')
plt.ylabel('Median Salary')
matched.groupby(['treat'])['re78'].median().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.subplot(133)
matched.groupby(['treat'])['diff'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.title('Difference of salary between 78 and 75 with/without training')
plt.ylabel('Difference of Salary')
plt.xlabel('Training')

matched = matched.drop('diff',1)

In [ ]:
f, axarr = plt.subplots(3, 3, figsize=(15, 18))
for index, feature in enumerate(['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']):
    ax = axarr[int(index/3)][index%3]
    plot_distrib(s1=matched[feature][matched['treat'] == 0], s2=matched[feature][matched['treat'] == 1], title=feature, xLabel = feature, yLabel='Density', ax=ax)

In [ ]:
#from scipy import stats


#for feature in ['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']:
    #ks = stats.ks_2samp(matched[feature][matched['treat'] == 0], matched[feature][matched['treat'] == 1])
    #print('For ' + str(feature)+ ' the KS stat and P-values are ' +str(ks))
    
from scipy import stats 
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\\n') 
for feature in ['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']: 
    ks = stats.ks_2samp(matched[feature][matched['treat'] == 0], matched[feature][matched['treat'] == 1]) 
    print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

## 5. Balancing the groups further
The "balanced" mode uses the values of y to automatically adjust
    weights inversely proportional to class frequencies in the input data
    as ``n_samples / (n_classes * np.bincount(y))`

In [ ]:
#lalonde = pd.read_csv('lalonde.csv')
#lal = lalonde.drop(['id','treat','re78'],1)
#lal = preprocessing.scale(lal)
#model = sklearn.linear_model.LogisticRegression()
#model.fit(lal, lalonde.treat)
#pred = model.predict_proba(lal)
#lalonde['pred'] = pred[:,1]


Gprime=nx.Graph()
Gprime.add_nodes_from(lalonde['id'][lalonde.treat == 0])
Gprime.add_nodes_from(lalonde['id'][lalonde.treat == 1])

lalonde_not_black = lalonde[lalonde.black == 0]
for ID_u, score_u in zip(lalonde_not_black.id[lalonde_not_black.treat == 0], lalonde_not_black.pred[lalonde_not_black.treat == 0]):
    for ID_v, score_v in zip(lalonde_not_black.id[lalonde_not_black.treat == 1], lalonde_not_black.pred[lalonde_not_black.treat == 1]):
        Gprime.add_edge(ID_u, ID_v, weight=-abs(score_u-score_v))

lalonde_black = lalonde[lalonde.black == 1]
for ID_u, score_u in zip(lalonde_black.id[lalonde_black.treat == 0], lalonde_black.pred[lalonde_black.treat == 0]):
    for ID_v, score_v in zip(lalonde_black.id[lalonde_black.treat == 1], lalonde_black.pred[lalonde_black.treat == 1]):
        Gprime.add_edge(ID_u, ID_v, weight=-abs(score_u-score_v))

In [ ]:
matching = max_weight_matching(Gprime, maxcardinality=True)

In [ ]:
lalonde.set_index('id' , inplace = True)
matched = lalonde.loc[list(matching.keys())]
matched

In [ ]:
f, axarr = plt.subplots(3, 3, figsize=(15, 18))
for index, feature in enumerate(['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']):
    ax = axarr[int(index/3)][index%3]
    plot_distrib(s1=matched[feature][matched['treat'] == 0], s2=matched[feature][matched['treat'] == 1], title=feature, xLabel = feature, yLabel='Density', ax=ax)

In [ ]:
from scipy import stats
print(('feature').ljust(10), ('statistic').ljust(10), ('p value').ljust(0), '\n')
for feature in ['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75', 're78']:
    ks = stats.ks_2samp(matched[feature][matched['treat'] == 0], matched[feature][matched['treat'] == 1])
    print(feature.ljust(10), '%.3f'.ljust(10) %ks[0], '%.3f'.ljust(0) %ks[1])

## 6. A less naive analysis

In [ ]:
matched['diff'] = matched['re78'] - matched['re75']



plt.figure(figsize=(20,5))
plt.subplot(131)
plt.title('Mean salary with/without training')
plt.ylabel('Mean salaray')
matched.groupby(['treat'])['re78'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.subplot(132)
plt.title('Median salary with/without training')
plt.ylabel('Median Salary')
matched.groupby(['treat'])['re78'].median().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.subplot(133)
matched.groupby(['treat'])['diff'].mean().plot.bar(color=[COLOR_NO_TREAT, COLOR_TREAT])
plt.title('Difference of salary between 78 and 75 with/without training')
plt.ylabel('Difference of Salary')
plt.xlabel('Training')

matched = matched.drop('diff',1)

# Question 2: Applied ML
We are going to build a classifier of news to directly assign them to 20 news categories. Note that the pipeline that you will build in this exercise could be of great help during your project if you plan to work with text!

- Load the 20newsgroup dataset. It is, again, a classic dataset that can directly be loaded using sklearn. TF-IDF, short for term frequency–inverse document frequency, is of great help when if comes to compute textual features. Indeed, it gives more importance to terms that are more specific to the considered articles (TF) but reduces the importance of terms that are very frequent in the entire corpus (IDF). Compute TF-IDF features for every article using TfidfVectorizer. Then, split your dataset into a training, a testing and a validation set (10% for validation and 10% for testing). Each observation should be paired with its corresponding label (the article category).

- Train a random forest on your training set. Try to fine-tune the parameters of your predictor on your validation set using a simple grid search on the number of estimator "n_estimators" and the max depth of the trees "max_depth". Then, display a confusion matrix of your classification pipeline. Lastly, once you assessed your model, inspect the feature_importances_ attribute of your random forest and discuss the obtained results.

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups = fetch_20newsgroups(subset='all')
newsgroups.target_names

Here are all the target names inside the newsgroup dataset
## 2.1 Load the data, vectorize it and split it into training set and training set

In [ ]:
from  sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(newsgroups.data) 
print(np.shape(X))

Our dataset has 18846 datapoints and 173762 features. Now let split it into train, valid and test set. To do so, we created a function *get_train_valid_test_set* written in the box below.

In [ ]:
def get_train_valid_test_set(data, labels, perc_for_valid, perc_for_test):
    # returns the train, valid and test sets
    # data : n x nb_features matrix
    # perc_for_valid : value between 0 and 1 as the percentage of the dataset to be used in validation
    # perc_for_test : value between 0 and 1 as the percentage of the dataset to be used in testing
    
    uindx = np.random.permutation(np.shape(data)[0])
    X_shuffled = data[uindx]
    labels_shf = labels[uindx]
    
    # take perc_for_test% of the dataset for testing
    X_test_shf = X_shuffled[:int(X_shuffled.shape[0]*perc_for_test)]
    y_test_shf = labels_shf[:int(X_shuffled.shape[0]*perc_for_test)]
    
    # take perc_for_valid% of the dataset for validation
    X_valid_shf = X_shuffled[int(X_shuffled.shape[0]*perc_for_test):
                             int(X_shuffled.shape[0]*(perc_for_test + perc_for_valid))]
    y_valid_shf = labels_shf[int(X_shuffled.shape[0]*perc_for_test):
                             int(X_shuffled.shape[0]*(perc_for_test + perc_for_valid))]
    
    X_train_shf = X_shuffled[int(X_shuffled.shape[0]*(perc_for_test + perc_for_valid)):]
    y_train_shf = labels_shf[int(X_shuffled.shape[0]*(perc_for_test + perc_for_valid)):]
    
    return X_test_shf, X_train_shf, X_valid_shf, y_test_shf, y_train_shf, y_valid_shf

In [ ]:
X_test, X_train, X_valid, y_test, y_train, y_valid = get_train_valid_test_set(X, newsgroups.target, 0.1, 0.1)
print('test set size : '+ str(X_test.shape))
print('valid set size : '+ str(X_valid.shape))
print('train set size : '+ str(X_train.shape))

## 2.2 Classification
**Train a random forest** on your training set. Try to **fine-tune the parameters** of your predictor on your validation set using a simple grid search on the number of estimator "n_estimators" and the max depth of the trees "max_depth". Then, **display a confusion matrix** of your classification pipeline. Lastly, once you assessed your model, **inspect the feature_importances_** attribute of your random forest and discuss the obtained results.

### Fine Tune max depth and number of trees

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

number_trees = [125, 300, 500, 600]
max_depth = [5, 8, 12, 16, 20]

best_score = 0

#Loop for hyperparameter number_trees and max_depth
for nb_t in number_trees:
    for nb_d in max_depth:

        # Random forest model
        rand_forest_model = RandomForestClassifier(n_estimators=nb_t, max_depth=nb_d)
        rand_forest_model.fit(X_train, y_train)
        y_pred_val = rand_forest_model.predict(X_valid)

        # get score
        score = accuracy_score(y_valid, y_pred_val)
        #print('loading : '+ str(stepinfo/(len(number_trees)*len(max_depth))) + '% nb_trees : ' + str(nb_t) + ' depth : '
        #      + str(nb_d) + ' score : ' + str(score))
        
        # update best score if needed
        if score > best_score:
            best_nb_trees = nb_t
            best_max_depth = nb_d
            best_score = score
print(best_score)       

### Display confusion matrix

In [ ]:
rand_forest_model = RandomForestClassifier(n_estimators=best_nb_trees, max_depth=best_max_depth)
rand_forest_model.fit(X_train, y_train)
y_pred_test = rand_forest_model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_pred_test, y_test)
#np.fill_diagonal(cm, 0)
plt.matshow(cm)
plt.title('Confusion matrix of the classifier')
plt.colorbar()
plt.show()


## TODO : regarder comment mettre la confusion matrix en % 
Faire la features importance
Analayser tout ca 

In [ ]:
print(best_nb_trees)
print(best_max_depth)

In [ ]:
print(y_pred_test)
sklearn.metrics.accuracy_score(y_pred_test, y_test)